In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
risangbaskoro_wlasl_processed_path = kagglehub.dataset_download('risangbaskoro/wlasl-processed')

print('Data source import complete.')


100%|██████████| 4.82G/4.82G [01:10<00:00, 73.3MB/s]

Extracting files...


Data source import complete.


In [3]:
pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 12.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
import os

path = risangbaskoro_wlasl_processed_path
main_df = pd.read_json(path + '/WLASL_v0.3.json')

def get_valid_ids(df):
    videos = []
    for instance in df:
        video_id = instance['video_id']
        if os.path.exists(f'{path}/videos/{video_id}.mp4'):
            videos.append(video_id)
    return videos

main_df['video_ids'] = main_df['instances'].apply(get_valid_ids)
print(main_df)

           gloss                                          instances  \
0           book  [{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra...   
1          drink  [{'bbox': [551, 68, 1350, 1080], 'fps': 25, 'f...   
2       computer  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...   
3         before  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...   
4          chair  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...   
...          ...                                                ...   
1995  washington  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...   
1996   waterfall  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...   
1997       weigh  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...   
1998  wheelchair  [{'bbox': [415, 86, 1811, 1080], 'fps': 25, 'f...   
1999     whistle  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...   

                                              video_ids  
0            [69241, 07069, 07068, 07070, 07099, 07074]  
1     [69302, 65539, 17710, 177

In [5]:
main_df = main_df.explode('video_ids', ignore_index=True)
main_df['paths'] = f'{path}/videos/' + main_df['video_ids'] + '.mp4'
main_df.columns = ['signs', 'instances', 'signer_ids', 'paths']

print(main_df)

            signs                                          instances  \
0            book  [{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra...   
1            book  [{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra...   
2            book  [{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra...   
3            book  [{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra...   
4            book  [{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra...   
...           ...                                                ...   
11975  wheelchair  [{'bbox': [415, 86, 1811, 1080], 'fps': 25, 'f...   
11976  wheelchair  [{'bbox': [415, 86, 1811, 1080], 'fps': 25, 'f...   
11977     whistle  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...   
11978     whistle  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...   
11979     whistle  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...   

      signer_ids                                              paths  
0          69241  /root/.cache/kagglehub/datasets/risangbaskoro/.

In [6]:
grouped = main_df.groupby('signs')
max_vids = 8
correct_group_list = []
for sign, group in grouped:
    if len(group) > max_vids:
        #updated = train_df.drop(group.sample(len(group) - max_vids).index)
        new_group = group.sample(max_vids)
        correct_group_list.append(new_group)
    else:
        correct_group_list.append(group)

updated = pd.concat(correct_group_list, ignore_index = True)

print(updated)

main = updated.copy()

          signs                                          instances signer_ids  \
0             a  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...      66039   
1             a  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...      01610   
2             a  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...      01612   
3             a  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...      01615   
4         a lot  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...      02128   
...         ...                                                ...        ...   
11547  yourself  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...      64448   
11548      zero  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...      64551   
11549      zero  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...      66825   
11550      zero  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...      64544   
11551      zero  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...      64547   

                           

In [7]:
import os
import cv2
import shutil
import time
import json

import mediapipe as mp
import numpy as np
import pandas as pd
import os
import shutil
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import glob
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
np.random.seed(42)
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F

In [8]:
import torch.nn.utils.rnn as rnn_utils
import torch.nn.functional as F
def pad_video(vid_features):
  max_len = 20
  num_frames, num_channels, height, width = vid_features.shape
  padded_features = F.pad(vid_features, (0, 0, 0, 0, 0, 0, 0, max_len - num_frames), value=0.0)
  return padded_features


In [9]:
def pad_frame(frame):
  resize = (256, 256)
  resized = cv2.resize(frame, resize)
  return resized


In [10]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*61*61, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size): # for now
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)

        out, (hidden, cell) = self.lstm(x, (h0, c0))

        out = self.fc(out[-1, :, :])  # Get the output from the last timeste

        return out

In [12]:
def vidToFeatures(path):

    features = []
    video_capture = cv2.VideoCapture(path)

    vid_cnt = 0
    success = True

    # Read the next frame
    success, frame = video_capture.read()

    frame_count = 0
    frame_int = 5
    while success and frame is not None:
        frame_count += 1
        if frame_count % frame_int == 0:
          frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          padded_frame = pad_frame(frame)
          height = padded_frame.shape[0]
          width = padded_frame.shape[1]
          dim_h = torch.full((1, 20, 1), height)
          dim_w = torch.full((1, 20, 1), width)

          frame_features = torch.tensor(padded_frame).permute(2, 0, 1)
          frame_features = frame_features.to(torch.float32)

          features.append(frame_features)

        success, frame = video_capture.read()

    video_capture.release()

    features_tensor = torch.stack(features)
    padded = pad_video(features_tensor)

    return padded



from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(main, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)



In [13]:
def evaluate(model_cnn, model_lstm, loader):
    model_cnn.eval()
    model_lstm.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for features, labels in loader:
            features, labels = features.to(device), labels.to(device)

            cnn_features = [model_cnn(frame) for frame in features]
            cnn_features = torch.stack(cnn_features).permute(1, 0, 2)

            outputs = model_lstm(cnn_features)
            _, predicted = torch.max(outputs, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    return 100 * correct / total

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Encode the labels to numeric indices
label_encoder = LabelEncoder()
label_encoder.fit(main_df['signs'])
train_df['encoded_signs'] = label_encoder.transform(train_df['signs'])
val_df['encoded_signs'] = label_encoder.transform(val_df['signs'])
test_df['encoded_signs'] = label_encoder.transform(test_df['signs'])

In [ ]:
# Modify the Dataset class to use encoded labels
class Dataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        path = self.dataframe.iloc[idx]['paths']
        label = self.dataframe.iloc[idx]['encoded_signs']
        features = vidToFeatures(path)
        return features, label

# Define the dataset
train_dataset = Dataset(train_df)
new_ds = Dataset(train_df[:2000])
val_dataset = Dataset(val_df)
test_dataset = Dataset(test_df)

# DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

new_loader = DataLoader(new_ds, batch_size=8, shuffle=False)

# Initialize models
cnn = CNN().to(device)
lstm = LSTMModel(128, 128, len(main_df['signs'].unique())).to(device)

# Loss function and optimizer
L = nn.CrossEntropyLoss()
optimizer = optim.Adam(list(cnn.parameters()) + list(lstm.parameters()), lr=0.001)

# Training loop

num_epochs = 5
train_acc =[]
val_acc = []
from itertools import islice

for epoch in range(num_epochs):
    cnn.train()
    lstm.train()
    total_loss = 0


    for i, stuff in enumerate(new_loader):
        features, labels = stuff
        features, labels = features.to(device), labels.to(device)
        print(features.shape)
        print(f'batch {i+1}')


        # Extract features with CNN
        cnn_features = []
        for frame in features:
            cnn_out = cnn(frame)
            cnn_features.append(cnn_out)
        cnn_features = torch.stack(cnn_features).permute(1, 0, 2)
        #print(cnn_features.shape)

        # Predict with LSTM
        outputs = lstm(cnn_features)
        #print(outputs.shape)
        loss = L(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}')

    train_accuracy = evaluate(cnn, lstm, train_loader)
    val_accuracy = evaluate(cnn, lstm, val_loader)
    train_acc.append(train_accuracy)
    val_acc.append(val_accuracy)



torch.Size([8, 20, 3, 256, 256])
batch 1
torch.Size([8, 20, 3, 256, 256])
batch 2
torch.Size([8, 20, 3, 256, 256])
batch 3
torch.Size([8, 20, 3, 256, 256])
batch 4
torch.Size([8, 20, 3, 256, 256])
batch 5
torch.Size([8, 20, 3, 256, 256])
batch 6
torch.Size([8, 20, 3, 256, 256])
batch 7
torch.Size([8, 20, 3, 256, 256])
batch 8
torch.Size([8, 20, 3, 256, 256])
batch 9
torch.Size([8, 20, 3, 256, 256])
batch 10
torch.Size([8, 20, 3, 256, 256])
batch 11
torch.Size([8, 20, 3, 256, 256])
batch 12
torch.Size([8, 20, 3, 256, 256])
batch 13
torch.Size([8, 20, 3, 256, 256])
batch 14
torch.Size([8, 20, 3, 256, 256])
batch 15
torch.Size([8, 20, 3, 256, 256])
batch 16
torch.Size([8, 20, 3, 256, 256])
batch 17
torch.Size([8, 20, 3, 256, 256])
batch 18
torch.Size([8, 20, 3, 256, 256])
batch 19
torch.Size([8, 20, 3, 256, 256])
batch 20
torch.Size([8, 20, 3, 256, 256])
batch 21
torch.Size([8, 20, 3, 256, 256])
batch 22
torch.Size([8, 20, 3, 256, 256])
batch 23
torch.Size([8, 20, 3, 256, 256])
batch 24
t

In [ ]:
val_accuracy = evaluate(cnn, lstm, val_loader)
print(f'Validation Accuracy: {val_accuracy:.2f}%')

test_accuracy = evaluate(cnn, lstm, test_loader)
print(f'Test Accuracy: {test_accuracy:.2f}%')

plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_acc, label='Training Accuracy')
plt.plot(range(1, num_epochs + 1), val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()